## Getting PBP

In [12]:
import pandas as pd
import numpy as np
import os
from scipy.stats import percentileofscore as perc
from simFootballPBP import *

'''
Functions
'''

def scoreDiff(row):
    if row['recTeam'] == row['awayTeam']:
        return row['homeScore'] - row['awayScore']
    else:
        return row['awayScore'] - row['homeScore']

def secSinceHalf(row):
    return 1800 - int(row['totTime'])

def fieldPosScore(row):
    if row['dist2goal'] >= 60:
        return 1.0
    elif row['dist2goal'] >= 50:
        return 1.1 ** (40 - row['dist2goal'])
    else:
        return 1.1**10 * 1.2**(50 - row['dist2goal'])

def distScore(row):
    if int(row['distance']) >= 10:
        return 0.2
    elif 7 <= row['distance'] <= 9:
        return 0.4
    elif 4 <= row['distance'] <= 6:
        return 0.6
    elif 2 <= row['distance'] <= 3:
        return 0.8
    else:
        return 1.0

def scoreDiffMult(pointdiff):
    if pointdiff > 0:
        return 1
    elif pointdiff == 0:
        return 2
    elif pointdiff < -8:
        return 3
    else:
        return 4

def timeMult(row):
    sec = secSinceHalf(row)
    PD = scoreDiff(row)
    
    if PD <= 0 and sec >= 0:
        return ((sec * 0.001)**3) + 1.
    else:
        return 1.

def surrenderIndex(row):
    pd = scoreDiff(row)
    return fieldPosScore(row) * distScore(row) * scoreDiffMult(pd) * timeMult(row)

def receiving(row):
    if row['teamPoss'] == row['homeTeam']:
        return row['awayTeam']
    else:
        return row['homeTeam']
    
def touchback(row):
    if 'Touchback' in row['play']:
        return "Touchback"
    elif 'No return' in row['play']:
        return "No return"
    else:
        return "Returned"

def puntDist(row):
    if 'BLOCKED' in row.play:
        return 0
    elif 'Touchback' in row.play:
        return row.dist2goal - 20
    else:
        return int(row.play.split('of ')[-1].split(' yards')[0])
    
def stringify(row):
    num = ['st','nd','rd','th']
    if row['awayTeam'] == row['teamPoss']:
        puntScore = row['awayScore']
        recScore = row['homeScore']
    else:
        puntScore = row['homeScore']
        recScore = row['awayScore']
    return "Q%s - %s - %i%s and %i\n%s %s - %s %s\n"%(row['Q'],row['time'],row['down'],num[int(row['down'])-1],row['distance'],row['teamPoss'],puntScore,row['recTeam'],recScore)


In [13]:
S = 25

sData = []
idList, idDict = getSeasonIDs(S)

print('Season:',S)
print('Games:',len(idList))

Season: 25
Games: 14


In [3]:
print('Gathering PBP...')
for i in idList:
    sData.append(getGameData(S,i,idDict))

sDF = pd.concat(sData)
sDF.to_csv('PBP/S%sPBP.csv'%S)

print('PBP for S%s saved!'%S)

Gathering PBP...
PBP for S25 saved!


In [5]:
print('Combining all Seasons...')
import os
allDF = pd.concat([pd.read_csv('PBP/%s'%p) for p in next(os.walk('PBP'))[2] if '.csv' in p])
allDF = allDF.reset_index().sort_values('S')
# allDF.to_csv('allPBP.csv')

print('Getting Punts...')
puntDF = allDF[allDF['play'].str.contains('Punt ')]

print('Done!')

Combining all Seasons...
Getting Punts...
Done!


In [6]:
print('Cleaning up PuntDF...')
NFL = np.load("2009-2018_surrender_indices.npy")
puntDF['recTeam'] = puntDF.apply(lambda row : receiving(row),axis=1)
puntDF['surrenderIndex'] = puntDF.apply(lambda row : surrenderIndex(row),axis=1)
puntDF = puntDF[['S','W','gameID','Q','time','awayTeam','awayScore','homeScore','homeTeam'
                 ,'teamPoss','recTeam','down','distance','side','dist2goal','surrenderIndex','play']]
puntDF = puntDF.sort_values('surrenderIndex',ascending=False)
puntDF['NFLpercentiles'] = puntDF.apply(lambda row : perc(NFL,row['surrenderIndex']),axis=1)
puntDF['percentiles'] = puntDF.apply(lambda row : perc(puntDF['surrenderIndex'],row['surrenderIndex']),axis=1)
puntDF['result'] = puntDF.apply(lambda row : touchback(row),axis = 1)
puntDF['scoreDiff'] = puntDF.apply(lambda row : scoreDiff(row),axis = 1)
puntDF['puntDist'] = puntDF.apply(lambda row : puntDist(row),axis = 1)
puntDF['puntEndLoc'] = puntDF['dist2goal'] - puntDF['puntDist']
puntDF['surrenderRank'] = puntDF.surrenderIndex.rank(method='max',ascending=False).astype('int')
puntDF['situation'] = puntDF.apply(lambda row : stringify(row), axis = 1)

puntDF = puntDF[['S', 'W', 'gameID', 'Q', 'time', 'awayTeam', 'awayScore', 'homeScore',
       'homeTeam', 'teamPoss', 'recTeam', 'down', 'distance', 'dist2goal', 'puntDist',
       'puntEndLoc', 'surrenderIndex','surrenderRank', 'percentiles', 'NFLpercentiles', 'play',
       'situation','result']]

print('Exporting Surrender DF...')
puntDF.to_csv('surrender.csv')

print('Done!')

Cleaning up PuntDF...


/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Exporting Surrender DF...
Done!


## Bot Functions

In [14]:
s25 = pd.read_csv('PBP/S25PBP.csv')

In [30]:
s25['play'].str.replace('Dasistwirklichseinnachname, A.','D, Alex')

0                                      D, Alex kicks off.
1                        The kick sails into the endzone.
2                PickSix, H. takes it out of the endzone!
3       Kickoff of 77 yards. Returned by PickSix, H. f...
4         Jack, M. SACKED by Rapid Eagle - DT for -7 yds.
                              ...                        
2928    Rush by Z, D. for 2 yds. Tackle by Annastesia,...
2929    Rush by Skyline, A. for 4 yds. Tackle by De Vi...
2930    Rush by Z, D. for a short gain. Tackle by Harr...
2931        Punt by Kokot (R), B. of 39 yards. Touchback.
2932    Pass by Fujiwara (C), C., complete to Howlett ...
Name: play, Length: 2933, dtype: object

In [17]:
test = s25[(s25.S == 25) & (s25.W == 1) & ((s25.homeTeam == 'COL') | (s25.awayTeam == 'COL'))]

In [26]:
string = test.play.iloc[0].split('yards. ')
print(str(string[0] + 'yards.\n' + string[1]))

Punt by Small, J. of 67 yards.
No return.


In [29]:
import tabulate
tabulate(test[['surrenderRank','situation', 'play','surrenderIndex','percentile']],
         headers=["Rank","Game Situation","Punt","Index","Perc."],tablefmt='psql',showindex=False)

KeyError: "['situation', 'percentile', 'surrenderIndex', 'surrenderRank'] not in index"

## S3

In [5]:
import boto3

s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  isfl-surrender-bot


In [8]:
%%timeit
s3.download_file('isfl-surrender-bot','surrender.csv','testsurrender.csv')

2.95 s ± 336 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
s3.upload_file('testsurrender.csv','isfl-surrender-bot','testsurrender.csv')